In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
print(tf.__version__)

2.7.0


# Download IMDB dataset


In [9]:
import requests
imdb = keras.datasets.imdb
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17473536/17464789 [==============================] - 1s 0us/step
